In [1]:
# Selenium 웹 자동화 관련 라이브러리
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from pathlib import Path

# 데이터 처리용
import pandas as pd
import time

In [ ]:
# 1) 웹드라이버 실행
driver = webdriver.Chrome()
driver.maximize_window()

# 2) 페이지 접속
url = "https://www.koreabaseball.com/Record/TeamRank/TeamRank.aspx"
driver.get(url)

# 수집할 연도/팀
years = [2020, 2021, 2022, 2023, 2024, 2025]

def find_year_select(driver):
    """
    KBO 페이지의 '연도 선택' 드롭다운을 최대한 안전하게 찾기 위한 함수.
    (id/name이 수업 때랑 약간 달라도 잡히게 넓게 탐색)
    """
    candidates = driver.find_elements(By.CSS_SELECTOR, "select")
    for el in candidates:
        try:
            sel = Select(el)
            opts = [o.text.strip() for o in sel.options]
            # 연도 옵션(2022/2023/2024/2025)이 들어있는 select를 찾는다
            if any(str(y) in opts for y in years):
                return sel
        except:
            pass
    raise RuntimeError("못찾음")

def scrape_one_year(year: int):
    # 연도 select 찾고 선택
    year_select = find_year_select(driver)
    year_select.select_by_visible_text(str(year))

    # 선택 후 테이블이 갱신될 시간을 조금 준다 (수업에서 sleep 써도 된다 했으니 최소로)
    time.sleep(1)

    # 테이블 행 가져오기
    rows = driver.find_elements(
        By.CSS_SELECTOR,
        "#cphContents_cphContents_cphContents_udpRecord > table > tbody > tr"
    )
    cols = ["rank", "team", "games", "wins", "losses", "draws", "win_rate",
            "gb", "last10", "streak", "home", "away"]

    data = []
    for r in rows:
        tds = r.find_elements(By.TAG_NAME, "td")
        if len(tds) < len(cols):
            continue

        row = {"year": year}
        for i, c in enumerate(cols):
            row[c] = tds[i].text.strip()

        data.append(row)

    return data

# 3)수집
all_data = []
for y in years:
    all_data.extend(scrape_one_year(y))

# 4) DataFrame 정리
win_df = pd.DataFrame(all_data)

# 숫자형으로 바꿀 컬럼들
num_int_cols = ["rank", "games", "wins", "losses", "draws"]
num_float_cols = ["win_rate", "gb"]

for c in num_int_cols:
    win_df[c] = (
        win_df[c].astype(str)
        .str.replace(r"[^0-9]", "", regex=True)
        .replace("", "0")
        .astype(int)
    )

for c in num_float_cols:
    win_df[c] = (
        win_df[c].astype(str)
        .str.replace(r"[^0-9.\-]", "", regex=True)  # 게임차에 '-' 있을 수 있으면 허용
        .replace("", "0")
        .astype(float)
    )

win_df = win_df.sort_values(["year", "rank"]).reset_index(drop=True)

# 브라우저 종료
driver.quit()


In [7]:
RAW_DIR = Path("../data/raw/kbo/team")
RAW_DIR.mkdir(parents=True, exist_ok=True)

save_path = RAW_DIR / "team_standings_basic_2020_2025.csv"
win_df.to_csv(save_path, index=False, encoding="utf-8-sig")